<a href="https://colab.research.google.com/github/james-lucius/qworld/blob/main/QB44_B02_Max_Cut_Problem_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://qworld.net" target="_blank" align="left"><img src="https://gitlab.com/qworld/qeducation/qbook101/raw/main/qworld/images/header.jpg" align="left"></a>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $
$ \newcommand{\greenbit}[1] {\mathbf{{\color{green}#1}}} $
$ \newcommand{\bluebit}[1] {\mathbf{{\color{blue}#1}}} $
$ \newcommand{\redbit}[1] {\mathbf{{\color{red}#1}}} $
$ \newcommand{\brownbit}[1] {\mathbf{{\color{brown}#1}}} $
$ \newcommand{\blackbit}[1] {\mathbf{{\color{black}#1}}} $

_prepared by Adam Glos_

<font size="28px" style="font-size:28px;" align="left"><b><font color="blue"> Solutions for </font>Max-Cut Problem</b></font>
<br>
<br><br>

##### <font color="#08b806">Please execute the following cell, it is necessary to distinguish between your local environment and Google Colab's

In [2]:
import IPython

def in_colab():
    try:
        import google.colab
        return True
    except:
        return False

if in_colab():
    !pip install cirq

<a name="task1"></a>
### Task 1
    
Let's implement the idea given above. We have a graph with 4 vertices, and so we have a circuit with 4 qubits.

Represent the following coloring of the given graph in the quantum circuit.

<img src="https://gitlab.com/qworld/qeducation/qbook101/raw/main/qbook101/images/ch4/graphcolor1.png" width="25%" align="center">

<h3> Solution </h3>

In [3]:
import cirq
from cirq import X, I

qq = cirq.LineQubit.range(4)
circuit = cirq.Circuit()

circuit.append(X(qq[1]))
circuit.append(X(qq[2]))

circuit.append(cirq.measure(*qq, key='result'))
print(circuit)

s = cirq.Simulator()
samples = s.run(circuit, repetitions=1)
result = samples.measurements["result"]

# decode the solution
for qubit in range(4):
    if result[0][qubit] == 0:
        print("Qubit",qubit, "is red")
    else:
        print("Qubit",qubit, "is blue")

0: ───────M('result')───
          │
1: ───X───M─────────────
          │
2: ───X───M─────────────
          │
3: ───────M─────────────
Qubit 0 is red
Qubit 1 is blue
Qubit 2 is blue
Qubit 3 is red


<a name="task2"></a>
### Task 2

For the given graph below, implement the first two steps of the oracle described above.

The first four qubits are used for vertices.

The next three qubits are used for edges.

The last qubit is used for the output.

Remark that the last qubit should be in state $ \ket{1} $ (resp., $\ket{0}$) if the coloring of the vertices is correct (resp., incorrect).

Test your implementation with different colorings.

<img src="https://gitlab.com/qworld/qeducation/qbook101/raw/main/qbook101/images/ch4/bipartite.png" width="25%" align="center">

<h3> Solution </h3>

In [4]:
import cirq
from cirq import X, CX
s = cirq.Simulator()

qq = cirq.LineQubit.range(8)
circuit = cirq.Circuit()
# correct coloring
circuit.append(X(qq[0]))
circuit.append(X(qq[1]))

# incorrect coloring
# circuit.append(X(qq[0]))

# check 0-2 edge and store at 4th qubit
circuit.append(CX(qq[0], qq[4]))
circuit.append(CX(qq[2], qq[4]))
# check 0-3 edge and store at 5th qubit
circuit.append(CX(qq[0], qq[5]))
circuit.append(CX(qq[3], qq[5]))
# check 1-3 edge and store at 6th qubit
circuit.append(CX(qq[1], qq[6]))
circuit.append(CX(qq[3], qq[6]))

# check all edge qubits
circuit.append(X(qq[7]).controlled_by(*(qq[4:7])))

circuit.append(cirq.measure(*qq, key='result'))
samples = s.run(circuit, repetitions=1)
result = samples.measurements["result"]
# 0 - first (and only measurement)
# 7 - last qubit
output = result[0][7]
print("Measurement output: ", output)
if output == 1:
    print("correct coloring (graph is bipartite)")
else:
    print("incorrect coloring")
print(circuit)

Measurement output:  1
correct coloring (graph is bipartite)
          ┌──┐   ┌──┐
0: ───X────@───────@────────────────M('result')───
           │       │                │
1: ───X────┼@──────┼────────────────M─────────────
           ││      │                │
2: ────────┼┼─────@┼────────────────M─────────────
           ││     ││                │
3: ────────┼┼─────┼┼────@───@───────M─────────────
           ││     ││    │   │       │
4: ────────X┼─────X┼────┼───┼───@───M─────────────
            │      │    │   │   │   │
5: ─────────┼──────X────X───┼───@───M─────────────
            │               │   │   │
6: ─────────X───────────────X───@───M─────────────
                                │   │
7: ─────────────────────────────X───M─────────────
          └──┘   └──┘


<a name="task3"></a>
### Task 3

For the given graphs below, iterate Grover’s search algorithm 2 steps to find the correct colorings. (There are indeed $k=2$ possible colorings, and so the oracle should be applied $\frac{\pi}{4}\sqrt{\frac{2^4}{k}}\approx 2$ times.)

You will use nine qubits: 4 for vertices, 4 for edges, and 1 for the output qubit.

The diffusion operator is provided below.

Observe which outcomes have the higher frequencies.


<img src="https://gitlab.com/qworld/qeducation/qbook101/raw/main/qbook101/images/ch4/completebipartite.png" width="25%" align="center">

<h3> Solution </h3>

In [5]:
import cirq
from cirq import X, H, Z, inverse, CX
s = cirq.Simulator()

qq = cirq.LineQubit.range(9)

def edge_check(a, b, c):
    yield CX(qq[a], qq[c])
    yield CX(qq[b], qq[c])

def oracle():
    # check 0-2 edge and store at 4th qubit
    yield edge_check(0, 2, 4)
    # check 0-3 edge and store at 5th qubit
    yield edge_check(0, 3, 5)
    # check 1-2 edge and store at 6th qubit
    yield edge_check(1, 2, 6)
    # check 1-3 edge and store at 7th qubit
    yield edge_check(1, 3, 7)

    # check all edge qubits
    yield X(qq[8]).controlled_by(*(qq[4:8]))

def oracle_computation():
    yield oracle()
    yield Z(qq[8])
    yield inverse(oracle())

def grover_diffusion():
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[3]).controlled_by(*(qq[0:3]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:4])))
for i in range(2):
    circuit.append(oracle_computation())
    circuit.append(grover_diffusion())

# we are only intertested in outputs of first 4 qubits
circuit.append(cirq.measure(*(qq[0:4]), key='result'))

# determine the statistics of the measurements
trials_number = 1000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

def bitstring(bits):
    return "".join(str(int(b)) for b in bits)

counts = samples.histogram(key="result",fold_func=bitstring)
print(counts)

Counter({'0011': 485, '1100': 467, '0101': 7, '1000': 6, '1001': 5, '1101': 4, '0010': 4, '1110': 4, '0001': 4, '1010': 3, '0111': 2, '1111': 2, '0110': 2, '0100': 2, '0000': 2, '1011': 1})


Note that the states '0011' and '1100' are the most commonly observed states.

In [6]:
print("Probability of measuring 0011: ", counts.get('0011')/trials_number)
print("Probability of measuring 1100: ", counts.get('1100')/trials_number)

Probability of measuring 0011:  0.485
Probability of measuring 1100:  0.467


<a name="task4"></a>
### Task 4

Repeat Task 3 for the following graph.

Does the following graph has any correct colorings or not? If not, what would you say in advance about the frequencies of the outcomes?


<img src="https://gitlab.com/qworld/qeducation/qbook101/raw/main/qbook101/images/ch4/nonbipartite.png" width="25%" align="center">

<h3> Solution </h3>

In [7]:
import cirq
from cirq import X, H, Z, inverse, CX
s = cirq.Simulator()

qq = cirq.LineQubit.range(9)

def oracle_computation():
    # check 0-1 edge and store at 4th qubit
    yield edge_check(0, 1, 4)
    # check 0-2 edge and store at 5th qubit
    yield edge_check(0, 2, 5)
    # check 0-3 edge and store at 6th qubit
    yield edge_check(0, 3, 6)
    # check 1-3 edge and store at 7th qubit
    yield edge_check(1, 3, 7)

    # check all edge qubits
    yield X(qq[8]).controlled_by(*(qq[4:8]))

def oracle():
    yield oracle_computation()
    yield Z(qq[8])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[3]).controlled_by(*(qq[0:3]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:4])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())

# we are only intertested in outputs of first 4 qubits
circuit.append(cirq.measure(*(qq[0:4]), key='result'))

# determine the statistics of the measurements
trials_number = 10000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)

counts = samples.histogram(key="result",fold_func=bitstring)
print(counts)

Counter({'1010': 661, '0110': 659, '0111': 650, '0011': 645, '1111': 645, '0010': 633, '1000': 633, '1110': 624, '0100': 621, '0101': 614, '0000': 613, '1101': 610, '0001': 605, '1001': 600, '1100': 596, '1011': 591})


We observe that all strings have almost the same occurence. The reason is that there is no correct coloring.